# Delta Lake Lab 
## Unit 8: Table Clone 

In the previous unit we-
1. Learned about zordering and data skipping native to delta lake

In this unit, we will learn about-
1. Table cloning - shallow clone; creation, and understanding of what happens when a shallow clone is created and when updated
2. Table cloning - deep clone; creation and understanding of what happens when a shallow clone is created and when updated 

### 1. Imports

In [1]:
import pandas as pd

from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

from delta.tables import *

import warnings
warnings.filterwarnings('ignore')

### 2. Create a Spark session powered by Cloud Dataproc 

In [2]:
spark = SparkSession.builder.appName('Loan Analysis').getOrCreate()
spark

22/10/22 23:44:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### 3. Declare variables

In [3]:
project_id_output = !gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID = project_id_output[0]
print("PROJECT_ID: ", PROJECT_ID)

PROJECT_ID:  delta-lake-lab


In [4]:
project_name_output = !gcloud projects describe $PROJECT_ID | grep name | cut -d':' -f2 | xargs
PROJECT_NAME = project_name_output[0]
print("PROJECT_NAME: ", PROJECT_NAME)

PROJECT_NAME:  delta-lake-lab


In [5]:
project_number_output = !gcloud projects describe $PROJECT_ID | grep projectNumber | cut -d':' -f2 | xargs
PROJECT_NUMBER = project_number_output[0]
print("PROJECT_NUMBER: ", PROJECT_NUMBER)

PROJECT_NUMBER:  885979867746


In [ ]:
ACCOUNT_NAME = "akhjain"

In [6]:
DATA_LAKE_ROOT_PATH= f"gs://dll-data-bucket-{PROJECT_NUMBER}-{ACCOUNT_NAME}"
DELTA_LAKE_DIR_ROOT = f"{DATA_LAKE_ROOT_PATH}/delta-consumable"
print(DELTA_LAKE_DIR_ROOT)

gs://dll-data-bucket-885979867746/delta-consumable


### 4. File listing

In [7]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.par

In [8]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT/part* | wc -l

30


In [9]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT/_delta_log/*.json | wc -l

13


### 5. Create a shallow clone

In [10]:
SHALLOW_CLONE_DIR = f"{DELTA_LAKE_DIR_ROOT}/shallow_clone/"
print(SHALLOW_CLONE_DIR)

gs://dll-data-bucket-885979867746/delta-consumable/shallow_clone/


In [11]:
spark.sql("SELECT * FROM loan_db.loans_by_state_delta WHERE addr_state='IA' LIMIT 2").show(truncate=False)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/spark/conf/ivysettings.xml will be used


+----------+-----+------------------+
|addr_state|count|collateral_value  |
+----------+-----+------------------+
|IA        |32   |326185.5741464369 |
|IA        |164  |1649795.5660065983|
+----------+-----+------------------+



In [12]:
spark.sql(f"CREATE TABLE IF NOT EXISTS loan_db.loans_by_state_delta_clone_shallow SHALLOW CLONE loan_db.loans_by_state_delta LOCATION \"{SHALLOW_CLONE_DIR}\"")

ParseException: 
Syntax error at or near 'SHALLOW'(line 1, pos 70)

== SQL ==
CREATE TABLE IF NOT EXISTS loan_db.loans_by_state_delta_clone_shallow SHALLOW CLONE loan_db.loans_by_state_delta LOCATION "gs://dll-data-bucket-885979867746/delta-consumable/shallow_clone/"
----------------------------------------------------------------------^^^


Shallow clone creation is a metadata operation until a CRUD operation is done against it, at which point, the data gets copy-persisted.

In [13]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.par

In [14]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT/part* | wc -l

30


In [15]:
spark.sql("UPDATE loan_db.loans_by_state_delta_clone_shallow SET count = 11111 WHERE addr_state='IL'")

AnalysisException: Table or view not found: loan_db.loans_by_state_delta_clone_shallow; line 1 pos 7;
'UpdateTable [assignment('count, 11111)], ('addr_state = IL)
+- 'UnresolvedRelation [loan_db, loans_by_state_delta_clone_shallow], [], false


In [16]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.par

In [17]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT/part* | wc -l

30


Because of the update, the data got copied over

### 6. Create a deep clone
A deep clone copies the data over.

In [19]:
DEEP_CLONE_DIR = f"{DELTA_LAKE_DIR_ROOT}/deep_clone/"
print(DEEP_CLONE_DIR)

gs://dll-data-bucket-885979867746/delta-consumable/deep_clone/


In [20]:
spark.sql(f"CREATE TABLE IF NOT EXISTS loan_db.loans_by_state_delta_clone_shallow DEEP CLONE loan_db.loans_by_state_delta LOCATION \"{DEEP_CLONE_DIR}\"")



ParseException: 
Syntax error at or near 'DEEP'(line 1, pos 70)

== SQL ==
CREATE TABLE IF NOT EXISTS loan_db.loans_by_state_delta_clone_shallow DEEP CLONE loan_db.loans_by_state_delta LOCATION "gs://dll-data-bucket-885979867746/delta-consumable/deep_clone/"
----------------------------------------------------------------------^^^


In [21]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT

gs://dll-data-bucket-885979867746/delta-consumable/:
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-0d0c8ea0-982f-4f67-ab9d-62f94e57db11-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-27993337-bc5b-4c93-9ab0-b77f48ac9160-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-293e9d10-a628-4cf0-b86c-f9f289913756-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-33f34593-184c-40b8-adfe-73facf9f043f-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-595b5ba1-408f-404d-91ee-7bc396235870-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-5c57d030-c1f7-4f7b-b5d8-2100c4426482-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-670d672a-c41d-4391-a47e-90d45e589ac2-c000.snappy.parquet
gs://dll-data-bucket-885979867746/delta-consumable/part-00000-8572a416-efe4-47bf-ab8b-755973ae5a7a-c000.snappy.par

In [22]:
!gcloud storage ls --recursive $DELTA_LAKE_DIR_ROOT/part* | wc -l

30


### THIS CONCLUDES THIS UNIT. PROCEED TO THE NEXT NOTEBOOK.